In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [3]:
pd.set_option("display.max_columns", 100)

In [4]:
analysis_date = dt.date(2022,5,25)

In [5]:
itp_id = 300

In [6]:
#route = 3479

In [7]:
df = (tbl.views.gtfs_schedule_fact_daily_trips()
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
      >> collect()
)

In [8]:
df

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,-4248367646246635494,883614,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,26520,27480,0.266667
1,4100255323842349957,4477524491535966539,883622,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,32760,34020,0.350000
2,4100255323842349957,6081166680947327825,882627,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,17,17,67140,68760,0.450000
3,4100255323842349957,-7852971863733032367,883349,3495,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,18,18,55500,59220,1.033333
4,4100255323842349957,-3711288289066616087,883357,3495,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,18,18,62700,66600,1.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,4100255323842349957,-4457574184755994218,881867,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,53,53,77400,80281,0.800278
1512,4100255323842349957,112618236911176424,881835,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,53,53,47040,51166,1.146111
1513,4100255323842349957,-8987643667339368721,881861,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,53,53,25200,28374,0.881667
1514,4100255323842349957,3218986837239881176,881831,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,53,53,50400,54814,1.226111


In [16]:
df.service_id.nunique()

6

In [17]:
df.service_id.value_counts()

10       1409
77710      52
40510      52
12010       1
92610       1
96910       1
Name: service_id, dtype: int64

In [9]:
(df
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> summarize(unique= _.trip_id.nunique())
      
)

,unique
0,1516


In [10]:
(df
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> group_by(_.route_id) 
 >> summarize(n=_.trip_id.nunique()) >> arrange(-_.n))

,route_id,n
0,3479,180
4,3485,153
8,3489,153
2,3481,145
9,3490,107
18,3501,104
12,3493,88
1,3480,82
15,3496,76
5,3486,71


In [21]:
# (tbl.views.gtfs_rt_fact_daily_feeds
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.date==analysis_date)
# )

In [11]:
trips =(tbl.views.gtfs_schedule_dim_trips()
 >>filter(_.calitp_itp_id==itp_id) >> collect())


In [12]:
trips

,calitp_itp_id,calitp_url_number,route_id,service_id,trip_id,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at,calitp_hash,trip_key,calitp_deleted_at
0,300,0,3344,10,860058,25372,Downtown SM Rapid,None,0,None,0,0,2021-09-01,9Pm/VjA6JvPzitIwMZ4CKA==,9219179982331851377,2021-10-11
1,300,0,3318,20210411_10,854557,25279,UCLA,None,0,None,0,0,2021-07-27,eyXuNwl4Ovjm1OV8mXSAFg==,-6937088259164875479,2021-09-01
2,300,0,3344,10,860062,25374,Aviation Sta/LAX Rapid,None,1,9305,0,0,2021-10-19,miUC+NxzdeZ+zqA7ye3rvQ==,830730113060854126,2021-10-21
3,300,0,3332,20210815_10,856792,25327,Palms Station,None,0,None,0,0,2021-07-27,DmZhbty6nPePiWn7W5Nwmw==,-7869173507519931822,2021-09-01
4,300,0,3335,10,857423,25340,Downtown SM,None,1,9803,0,0,2021-10-19,x7eyti4W1BKp3uVvlAuvQA==,-7560121890518539229,2021-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41265,300,0,3490,10,882854,25921,Playa Vista,None,1,None,0,0,2022-03-29,3bBbHJtgrZ+sVAGDWmT61A==,827868094658809629,2099-01-01
41266,300,0,3490,1,882721,25921,Playa Vista,None,1,None,0,0,2022-03-29,+k0DpFqBNlBI/w/yjbbdcA==,3365752046484738586,2099-01-01
41267,300,0,3490,10,882874,25921,Playa Vista,None,1,None,0,0,2022-03-29,fjqMOemnX+f7aBuMZz/2VQ==,-3499254430860761870,2099-01-01
41268,300,0,3490,10,882878,25921,Playa Vista,None,1,None,0,0,2022-03-29,XCUca0+rVJ0Fy9bDk4xEsg==,2124542854886673800,2099-01-01


In [18]:
#pd.merge(df, trips, on="route_id")

In [8]:
# (tbl.views.gtfs_schedule_fact_daily_trips()
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.service_date == analysis_date)
# )

In [9]:
# (
#     tbl.views.gtfs_rt_fact_daily_feeds()
#     >> filter(_.calitp_itp_id == itp_id)
#     >> arrange(_.date ==analysis_date)
# )

In [10]:
# airtable_organizations = (
#     tbl.airtable.california_transit_organizations()
#     >> select(_.itp_id, _.name, _.caltrans_district,
#               _.website, _.ntp_id, _.drmt_organization_name)
#     >> filter(_.itp_id == itp_id)
#     >> collect()
# )

In [11]:
# airtable_organizations